In [1]:
# LIBRARY IMPORTS
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam

In [2]:
# DATASET DIRECTORY CONFIGURATION

# Define the paths to the training and testing datasets
# Ensure the dataset is properly downloaded and unzipped before running the code
test_dir = r"C:\Users\USER\OneDrive\Desktop\Codes\muffin-vs-chihuahua\test"
train_dir = r"C:\Users\USER\OneDrive\Desktop\Codes\muffin-vs-chihuahua\train"

In [3]:
# IMAGE PARAMETERS

# Define the image size for resizing and the batch size for data loading
IMG_SIZE = (224, 224)  # Resize all images to 224x224 pixels for ResNet50
BATCH_SIZE = 32  # Number of images to process in a batch

In [4]:
# DATA PREPROCESSING & AUGMENTATION

# Apply data augmentation to the training dataset to improve model generalization
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=15,  # Randomly rotate images by up to 15 degrees
    width_shift_range=0.1,  # Randomly shift images horizontally by 10%
    height_shift_range=0.1,  # Randomly shift images vertically by 10%
    horizontal_flip=True,  # Randomly flip images horizontally
    validation_split=0.2  # Reserve 20% of training data for validation
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Only normalize test data

# Create data generators for training, validation, and testing datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',  # Use the training subset
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',  # Use the validation subset
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False  # Do not shuffle test data to maintain order
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [5]:
# RESNET50 MODEL ARCHITECTURE
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)
optimizer = Adam(learning_rate=lr_schedule)
IMG_SIZE = (224, 224)  # ResNet50 default input size
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
# Freeze the ResNet50 base layers
for layer in resnet_base.layers:
    layer.trainable = False
# Add custom layers on top of ResNet50
x = Flatten()(resnet_base.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(1, activation='sigmoid')(x)  # Binary output
model = Model(inputs=resnet_base.input, outputs=output_layer)

In [6]:
# CONFIGURE THE MODEL
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [7]:
# TRAINING THE RESNET MODEL
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 217s 2s/step - accuracy: 0.6360 - loss: 1.3935 - val_accuracy: 0.6878 - val_loss: 0.5551
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.6573 - loss: 0.6007 - val_accuracy: 0.7524 - val_loss: 0.5741
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 179s 2s/step - accuracy: 0.6983 - loss: 0.5811 - val_accuracy: 0.7556 - val_loss: 0.4999
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 177s 1s/step - accuracy: 0.6940 - loss: 0.5823 - val_accuracy: 0.8063 - val_loss: 0.4767
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.7210 - loss: 0.5665 - val_accuracy: 0.8212 - val_loss: 0.4476
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.6856 - loss: 0.5622 - val_accuracy: 0.7937 - val_loss: 0.4943
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.6534 - loss: 0.5936 - val_accuracy: 0.7270 - val_loss: 0.5448
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 713s 6s/step - accuracy: 0.6581 - loss: 0.5894 - val_accu

In [8]:
# EVALUATE THE RESNET MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.4f}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 156s 4s/step - accuracy: 0.8218 - loss: 0.4361
Test accuracy: 0.8218


In [9]:
# SAVE THE RESNET MODEL
model.save('exercise_6_arostique_ResNet.h5')
print("Model saved as 'exercise_6_arostique_ResNet.h5'")

Model saved as 'exercise_6_arostique_ResNet.h5'


In [10]:
# SIMPLE INFERENCE SCRIPT

from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_arostique_ResNet.h5'):
    # Load the trained model
    model = tf.keras.models.load_model(model_path)

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))  # Ensure this matches model input
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make a prediction
    pred = model.predict(img_array)[0, 0]
    label = "Muffin" if pred >= 0.5 else "Chihuahua"
    print(f"Prediction: {label} (confidence: {pred:.2f})")

In [11]:
# EXAMPLE USAGE

# Test the model with example images
predict_image(r"C:\Users\USER\OneDrive\Desktop\Codes\muffin-vs-chihuahua\run\run_1.jpeg")
predict_image(r"C:\Users\USER\OneDrive\Desktop\Codes\muffin-vs-chihuahua\run\run_2.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Prediction: Chihuahua (confidence: 0.33)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Prediction: Muffin (confidence: 0.68)
